# **AI Doctor: Virtual Symptom Checker and Treatment Guide**

AI Doctor is a virtual healthcare assistant powered by AI, designed to help users check their symptoms, receive possible diagnoses, and get medication suggestions based on their input. It also allows users to upload medical reports (PDFs, images, or CSV files) for further analysis. This project uses Generative AI, natural language processing, and OCR technologies to offer an interactive and user-friendly healthcare experience.


# Install necessary Libraries

In [138]:
!pip install google-generativeai


In [159]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.9 MB/s eta 0:00:00


In [161]:
!pip install pytesseract

# Import Required libraries

In [175]:
import os
import google.generativeai as genai
import PyPDF2
import pandas as pd
import pytesseract
from PIL import Image
from google.colab import files

* os - For file path handling
*  google.generativeai as genai  - For interacting with Google's generative AI model
* PyPDF2  - For reading and extracting text from PDF files
* pandas as pd - For handling CSV data
* pytesseract -  For Optical Character Recognition (OCR) to extract text from images
* from PIL import Image -  For handling image files and OCR processing


In [198]:
# Install Tesseract OCR software, required by pytesseract
!apt-get install tesseract-ocr


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 18 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 0s (13.3 MB/s)
Selecting previously unselected package tesseract-ocr-eng.
(Reading database ... 124926 files and directories currently installed.)
Preparing to unpack .../tesseract-ocr-e

In [199]:
# Set the path for the Tesseract OCR executable to enable text extraction from images
import pytesseract
from PIL import Image

# Make sure Tesseract is installed and provide the path to it
pytesseract.pytesseract.tesseract_cmd = '/usr/bin/tesseract'


In [208]:
#Configure Google Generative API key
genai.configure(api_key="Use API KEY")



In [209]:
# Model Configuration
generation_config = {
  "temperature": 0.5, # Controls the randomness of the output (0.0 to 1.0)
  "top_p": 0.95, # Top-p sampling for controlling diversity in responses
  "top_k": 40, # Top-k sampling to limit response options
  "max_output_tokens": 8192,  # Maximum number of tokens (words or word pieces) to generate
}

model = genai.GenerativeModel(
  model_name="gemini-1.5-pro-latest",  # Use the latest version for potentially better performance
  generation_config=generation_config,
)
# Start a chat session with the model to hold a conversation
chat_session = model.start_chat(history=[])

In [210]:
# Function to get the response from the model
def get_gemini_response(prompt):
  response = chat_session.send_message(prompt) # Send prompt to the model
  return response.text  # Return the model's response as text


# Images or pdf extraction


In [211]:
def extract_pdf_text(file_path):
    try:
        with open(file_path, 'rb') as file:  # Open PDF file in read-binary mode
            reader = PyPDF2.PdfReader(file)  # Read the PDF
            text = ""
            for page in reader.pages:  # Loop through each page of the PDF
                text += page.extract_text()  # Extract text from each page
        return text  # Return the extracted text
    except Exception as e:  # Handle any exceptions (e.g., file errors)
        return str(e)  # Return the error message


In [212]:
# Function to extract text from images (e.g., scanned reports) using pytesseract
def extract_image_text(file_path):
    try:
        img = Image.open(file_path)  # Open the image file
        text = pytesseract.image_to_string(img)  # Extract text using OCR
        return text  # Return the extracted text
    except Exception as e:  # Handle errors
        return str(e)  # Return error message


In [213]:
# Function to extract data from CSV files using pandas
def extract_csv_data(file_path):
    try:
        df = pd.read_csv(file_path)  # Read the CSV file into a DataFrame
        return df.head()  # Return the first few rows of the data (for quick preview)
    except Exception as e:  # Handle errors
        return str(e)  # Return the error message


# Symptom check

In [214]:
# Symptom Checker - Function to interact with the user and suggest possible diagnoses
def symptom_checker(symptoms_input):
    prompt = f"""Given the following symptoms: {symptoms_input},
                 suggest possible medical conditions. After listing possible conditions and severe symptoms,
                 **engage the user by asking 3-5 specific, targeted questions** to gather more information about their symptoms and health history.
                 This interaction should help to narrow down the potential diagnoses.
                 Phrase the questions in a friendly, professional, and empathetic tone, as if a doctor were speaking directly to the patient.
                 Conclude by emphasizing the importance of monitoring symptoms and seeking medical attention if necessary.
                 Use markdown **headings** and **underlining** for highlighting key points.
                 Add relevant emojis to make the conversation more engaging 😊.
                 """
    response = get_gemini_response(prompt) # Send the symptom details to the model
    return response # Return the model's diagnosis response


# Prescribe medication or treatment based on the diagnosis

In [215]:
# Prescribe Medication - Function to suggest medications based on diagnosis
def prescribe_medication(diagnosis):
    prompt = f"""Based on the diagnosis: {diagnosis},
                 suggest possible medications, treatments, and any lifestyle changes.
                 **Please list the treatments in bullet points**,
                 **underline any critical medication**, and summarize the next steps for the patient.
                 Add emojis to represent the mood (e.g.,💊 for medication, 🏥 for hospital visits).
                 """
    response = get_gemini_response(prompt)  # Send the diagnosis to the model for medication suggestions
    return response  # Return the medication or treatment suggestions

# Ask the user follow-up questions to gather more symptoms or concerns

In [216]:
# Ask Follow-Up - Function to interact with the user for follow-up questions
def ask_follow_up():
    follow_up_prompt = f"""Can you share more symptoms or ask any questions related to your health?
                           I'm here to help. I can also summarize any treatments or diagnoses.
                           **Please respond with more details or follow-up questions**.
                           Add friendly emojis like 🤔 for pondering and 🩺 for medical-related questions.
                           """
    response = get_gemini_response(follow_up_prompt) # Send the follow-up prompt to the model
    return response # Return the model's follow-up response

# Simulate the process of uploading a medical report file (PDF, image, or CSV)

In [217]:
# Function to simulate a file upload process
def upload_report():
    uploaded = files.upload()  # Open file upload dialog in Google Colab
    if uploaded:  # If the file is uploaded successfully
        file_path = list(uploaded.keys())[0] # Get the file path of the uploaded file
        return file_path # Return the file path
    return None # Return None if no file was uploaded



# Main healthcare system function to drive the workflow

In [218]:
# Main healthcare system function
def run_healthcare_system():
    print("# Welcome to the AI Doctor! 🩺")  # Print welcome message
    print("I'm here to assist you with your health-related questions. 🤖💬\n")


    # Collect symptoms from the user for the symptom checker
    symptoms_input = input("Please describe your symptoms (e.g., fever, cough, sore throat): ")
    diagnosis = symptom_checker(symptoms_input)  # Get potential diagnosis based on symptoms
    print(f"\n### Diagnosis: {diagnosis} 🤒\n")  # Display diagnosis result


    # Provide medication or treatment suggestions based on the diagnosis
    prescriptions = prescribe_medication(diagnosis) # Get medication suggestions
    print(f"\n### Suggested Medications/Treatments: {prescriptions} \n")  # Display medication


    # Ask the user follow-up questions about their symptoms or condition
    follow_up = ask_follow_up()  # Get follow-up questions
    print(follow_up) # Display follow-up responses



    # Simulate file upload to analyze the medical report
    print("Please upload your medical report.")
    file_path = upload_report() # Upload the medical report

    if file_path: # If a file was uploaded successfully
        # Extract text from the uploaded report based on the file type
        if file_path.lower().endswith(".pdf"):
            report_text = extract_pdf_text(file_path)    # Extract text from PDF
        elif file_path.lower().endswith(('.png', '.jpg', '.jpeg')):
            report_text = extract_image_text(file_path)  # Extract text from image
        elif file_path.lower().endswith(".csv"):
            report_text = extract_csv_data(file_path)    # Extract data from CSV
        else:
            print("Unsupported file format. Please try again with a valid report format.")
            return

        if report_text:  # If text was successfully extracted from the report
            print("\n### Analyzing your report... 🩸\n")   # Get analysis from the model
            diagnosis_from_report = get_gemini_response(f"Analyze the following medical report and provide a diagnosis: {report_text}")
            print(f"### Report Diagnosis and Recommendations: {diagnosis_from_report} 🤒💊\n")  # Display analysis result

        else:
            print("Failed to extract data from the report. Please check the file and try again. 📄🚫\n")  # If extraction fails
    else:
        print("No file selected. Please try again. ❌")   # If no file is uploaded


    # Allow the user to ask more questions or symptoms
    while True:
        more_questions = input("Do you have any more questions or symptoms to discuss? (yes/no): ").strip().lower()
        if more_questions == 'yes':
            chat_input = input("Please ask your question: ")  # User asks a new question
            response = get_gemini_response(chat_input)        # Get the response from the model
            print(f"\n### AI Doctor: {response} 🤖🩺\n")     # Display the response
        elif more_questions == 'no':
            print("Thank you for using the AI Doctor. Stay healthy! 😊👋")   # Thank the user if no more question
            break  # End the loop
        else:
            print("Please respond with 'yes' or 'no'.")

In [220]:
# Run the healthcare system
if __name__ == "__main__":
    run_healthcare_system() # Start the healthcare system function

# Welcome to the AI Doctor! 🩺
I'm here to assist you with your health-related questions. 🤖💬

Please describe your symptoms (e.g., fever, cough, sore throat): Throat infection

### Diagnosis: ## Possible Medical Conditions Related to Throat Infection 🤔

A throat infection can manifest in various ways, and several conditions could be the underlying cause:

* **Pharyngitis (Sore Throat):** This is a common inflammation of the pharynx, often caused by viral or bacterial infections.
* **Tonsillitis:**  Inflammation and infection of the tonsils, often caused by bacteria like Streptococcus (strep throat).
* **Laryngitis:** Inflammation of the larynx (voice box), often leading to hoarseness or voice loss.
* **Epiglottitis:** A rare but serious infection of the epiglottis (the flap of tissue that covers the windpipe), which can obstruct breathing.
* **Mononucleosis (Mono):**  Often called the "kissing disease," this viral infection can cause a severe sore throat, fatigue, and swollen lymph node

Saving nejmcpc1406191_t1.jpg to nejmcpc1406191_t1.jpg

### Analyzing your report... 🩸

### Report Diagnosis and Recommendations: The lab results suggest an **anemia** along with an active inflammatory process.  Let's break down the key findings:

* **Low Hematocrit (28.4% then 27.3%):**  This indicates a reduced percentage of red blood cells in the blood, a key feature of anemia. The decreasing value is concerning and suggests ongoing blood loss or reduced production.
* **Low Hemoglobin (9.3 g/dl then 8.8 g/dl):**  Hemoglobin carries oxygen in red blood cells.  Low levels confirm the anemia and may explain symptoms like fatigue and shortness of breath. Again, the decline is worrisome.
* **Smear Description:**  The presence of ovalocytes, hypochromasia (pale red blood cells), microcytosis (small red blood cells), and anisocytosis (variation in red blood cell size) further supports the diagnosis of anemia and suggests a possible issue with red blood cell production.  Further investigatio